In [ ]:
import pandas as pd
import sys
import os
import numpy 
import pylab as pl

import json
import os
s = json.load( open(os.getenv('PUI2015')+"/fbb_matplotlibrc.json") )
pl.rcParams.update(s)



%pylab inline

In [1]:
#to use the url just pas it to pandas read_excell
url="http://cosmo.nyu.edu/~fb55/UI_CUSP_2015/data/"
#split your data by male and female, for example you can load the data into 2 disctionary. other variables may be relevant: for example race.
#the names are not informative, and i am not giving you any meta data to find out what is in each file. 
# 08_2 files are male, 08_3 are female, and I am leaving in the lines of code that read the male files into races

allmales={}
allfemales={}
allmales['all']=pd.read_excel(url+'/pinc08_2_1_1.xls',header=9)#%s/pinc03_2_1_1.xls'%url, header=9)
allmales['white']=pd.read_excel(url+'/pinc08_2_1_4.xls', header=9)
allmales['black']=pd.read_excel(url+'/pinc08_2_1_6.xls', header=9)
allmales['hispanic']=pd.read_excel(url+'/pinc08_2_1_9.xls', header=9)
allmales['asian']=pd.read_excel(url+'/pinc08_2_1_8.xls', header=9)

allfemales['all']=pd.read_excel(url+'/pinc08_2_1_1.xls',header=9)#%s/pinc03_2_1_1.xls'%url, header=9)
allfemales['white']=pd.read_excel(url+'/pinc08_2_1_4.xls', header=9)
allfemales['black']=pd.read_excel(url+'/pinc08_2_1_6.xls', header=9)
allfemales['hispanic']=pd.read_excel(url+'/pinc08_2_1_9.xls', header=9)
allfemales['asian']=pd.read_excel(url+'/pinc08_2_1_8.xls', header=9)

NameError: name 'pd' is not defined

In [ ]:
#lest see what is in the data
from pandas.tools.plotting import scatter_matrix
#scatter_matrix(allmales, s=300, figsize=(16, 16), diagonal='kde')

#like for the building homework: this fails cause the data is not parsed and formatted propoerly
#so maybe just print the columns to find out
print allmales['all'].columns

In [ ]:
for x in allmales:
    nonreal  = allmales[x].applymap(np.isreal)
    for c in allmales[x].columns :
        if nonreal.sum()[c]>1 and nonreal.sum()[c]<49:
            allmales[x][c][~nonreal[c]] = np.float('NaN')
            allmales[x][c] = allmales[x][c].astype(float)

for k in allfemales:
    nonreal  = allfemales[k].applymap(np.isreal)
    for c in allfemales[k].columns :
        if nonreal.sum()[c]>1 and nonreal.sum()[c]<49:
            allfemales[k][c][~nonreal[c]] = np.float('NaN')
            allfemales[k][c] = allfemales[k][c].astype(float)

In [ ]:
males = []
females = []
for n in sorted(males.keys()):
    males.append(allmales[n]['Median income'].sum())
for n in sorted(allfemales.keys()):
    females.append(allfemales[n]['Median income'].sum())
x = maleList
y = femaleList
pl.scatter(x,y,'x',color='r')
pl.xlabel('Male income')
pl.ylabel('Female income')

Female values are less than their corresponding male values...70 for males = 55 for females.

In [ ]:
def line(var,m,b):
    return m*var+b

In [ ]:

#fit and plot here
#[...]
xi = np.arange(100000,800000,5000)
m, n = np.polyfit(x,y,1)
mod = line(xi,m,n)
print m,n
pl.plot(xi,mod)
pl.plot([100000,800000],[100000,800000])
pl.legend(['female vs male','fitted line'],loc=2)



In [ ]:


#this is the analytical best regression line. it can be proved (ask me about it if you want!)
from scipy import stats
all_males = np.asarray(maleList)
all_females = np.asarray(femaleList)
cor = stats.spearmanr(all_males, all_females)
all_males.std()/all_females.std()
beta1=cor[0]*all_females.std()/all_males.std()
beta0=all_females.mean()-beta1*all_males.mean()



In [ ]:
#[...]
moremod = line(xi,beta1,beta0)
pl.plot(xi,moremod)
pl.plot([100000,800000],[100000,800000])
pl.legend(['female vs male','fitted line'],loc=2)

In [ ]:
all_males=np.ma.array(np.concatenate([allmales['all']['Median income'][:].values,allmales['white']['Median income'][:].values,allmales['black']['Median income'][:].values,allmales['hispanic']['Median income'][:].values,allmales['asian']['Median income'][:].values]).flatten())

all_females=np.ma.array(np.concatenate([allfemales['all']['Median income'][:].values,allfemales['white']['Median income'][:].values,allfemales['black']['Median income'][:].values,allfemales['hispanic']['Median income'][:].values,allfemales['asian']['Median income'][:].values]).flatten())

#I am gonna create the mask int he next line. 
#the mask is True where either male or female value is Nan. 
#Notice that True is really just >1, and False really 0. 
#so i can add the male and female mask and the resulting mask will be 
#False where both arrays are not Nan, and true otherwise. 

allmask = [np.isnan(all_males) + np.isnan(all_females)]

all_females.mask = allmask
all_males.mask = allmask

In [ ]:
#fit and plot [...]
all_males = all_males[~all_males.mask]
all_females = all_females[~all_females.mask]
#print len(all_males)
#print len(all_females)
pl.plot(all_males,all_females,'x')
a, b = np.polyfit(all_males,all_females,1)
maskmod = line(all_males,a,b)
print a,b
pl.plot(all_males,maskmod)

Fit is reasonably similar with lots of high leverage points. you would just enter your variable stipend as x in your preferred model to receive the female equivalent. You could then pull out models by race, and do the same thing i.e. allMales['<race>'] vs allFemales[',race.']